In [43]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential
from keras import layers

In [44]:
df = pd.read_csv(r'D:\Fake News Detection\Data\data.csv') 

In [45]:
#df.drop(['stt','id','timestamp_post'], axis='columns', inplace=True)

In [46]:
#inputs = df.drop('label',axis='columns')
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()
corpus = []
for i in range(0, len(df)):
    review = df.post_message[i]
    review = review.split()
    review = ' '.join(review)
    corpus.append(review)
#inputs.head()

In [47]:
one_hotrep = [one_hot(word, 50000) for word in corpus]
padding = pad_sequences(one_hotrep, maxlen = 25, padding = 'pre')
#one_hotrep

In [48]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(50000,embedding_vector_features,input_length=25))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [49]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 25, 40)            2000000   
_________________________________________________________________
dropout_15 (Dropout)         (None, 25, 40)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_16 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                6464      
_________________________________________________________________
dropout_17 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                

In [50]:
from sklearn.model_selection import train_test_split
inputs = np.array(padding)
label = np.array(df.label)
X_train, X_test, y_train, y_test = train_test_split(inputs, label, test_size = 0.2)
X_train.shape

(2713, 25)

In [51]:
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 10, batch_size = 64)

Epoch 1/10
43/43 [==============================] - 3s 60ms/step - loss: 0.5321 - accuracy: 0.8161 - val_loss: 0.4605 - val_accuracy: 0.8277
Epoch 2/10
43/43 [==============================] - 2s 43ms/step - loss: 0.4574 - accuracy: 0.8242 - val_loss: 0.4724 - val_accuracy: 0.8277
Epoch 3/10
43/43 [==============================] - 2s 42ms/step - loss: 0.4095 - accuracy: 0.8253 - val_loss: 0.5569 - val_accuracy: 0.8218
Epoch 4/10
43/43 [==============================] - 2s 42ms/step - loss: 0.3081 - accuracy: 0.8703 - val_loss: 0.6201 - val_accuracy: 0.7644
Epoch 5/10
43/43 [==============================] - 2s 44ms/step - loss: 0.2358 - accuracy: 0.9049 - val_loss: 0.7364 - val_accuracy: 0.7393
Epoch 6/10
43/43 [==============================] - 2s 52ms/step - loss: 0.1672 - accuracy: 0.9381 - val_loss: 0.8593 - val_accuracy: 0.7541
Epoch 7/10
43/43 [==============================] - 2s 53ms/step - loss: 0.1330 - accuracy: 0.9506 - val_loss: 0.9569 - val_accuracy: 0.7791
Epoch 8/10
43

In [52]:
y_pred = model.predict_classes(X_test)
metrics.accuracy_score(y_test, y_pred)

0.727540500736377

In [53]:
#print('Training size = %d, accuracy = %.2f%%\n' % \
#     (X_train.shape[0], accuracy_score(y_test, y_pred)*100))
#print('Confusion matrix:\n', metrics.confusion_matrix(label_test, label_pred))
#print(metrics.classification_report(label_test, label_pred))